<a href="https://colab.research.google.com/github/JakeOh/202105_itw_bd26/blob/main/lab_da/da13_reshape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFrame 모양(shape) 변경

* wide (columns) --> long (rows)
* long (rows) ---> wide (columns)

In [1]:
import numpy as np
import pandas as pd

# stack vs unstack

In [4]:
df = pd.DataFrame(data=np.arange(1, 7).reshape((2, 3)),
                  columns=['a', 'b', 'c'], 
                  index=['A', 'B'])
df

,a,b,c
A,1,2,3
B,4,5,6


In [5]:
df_stacked = df.stack()
df_stacked  #> 컬럼 이름들이 index로 바뀜

A  a    1
   b    2
   c    3
B  a    4
   b    5
   c    6
dtype: int64

In [6]:
df_stacked.unstack()  #> 가장 마지막 레벨의 인덱스가 컬럼 이름으로 바뀜.

,a,b,c
A,1,2,3
B,4,5,6


In [7]:
df_stacked.unstack(level=0)  #> 첫번째 레벨의 인덱스가 컬럼 이름으로 바뀜.

,A,B
a,1,4
b,2,5
c,3,6


In [9]:
df = pd.DataFrame(data=np.arange(1, 13).reshape((2, 6)),
                  columns=[['Lunch'] * 3 + ['Dinner'] * 3, 
                           ['Fri', 'Sat', 'Sun'] * 2])
df

Lunch         Dinner        
    Fri Sat Sun    Fri Sat Sun
0     1   2   3      4   5   6
1     7   8   9     10  11  12

In [10]:
df_stacked1 = df.stack()  #> 가장 마지막 레벨의 컬럼 이름들을 index로 변환
df_stacked1

Dinner  Lunch
0 Fri       4      1
  Sat       5      2
  Sun       6      3
1 Fri      10      7
  Sat      11      8
  Sun      12      9

In [11]:
df_stacked1.unstack()  #> 가장 마지막 레벨의 인덱스를 컬럼 이름들로 변환

Dinner         Lunch        
     Fri Sat Sun   Fri Sat Sun
0      4   5   6     1   2   3
1     10  11  12     7   8   9

In [12]:
df_stacked2 = df.stack(level=0)
df_stacked2

Fri  Sat  Sun
0 Dinner    4    5    6
  Lunch     1    2    3
1 Dinner   10   11   12
  Lunch     7    8    9

In [13]:
df_stacked2.unstack()

Fri          Sat          Sun      
  Dinner Lunch Dinner Lunch Dinner Lunch
0      4     1      5     2      6     3
1     10     7     11     8     12     9

# pivot vs melt

In [15]:
df = pd.DataFrame({
    'time': ['Lunch'] * 3 + ['Dinner'] * 3,
    'day': ['Fri', 'Sat', 'Sun'] * 2,
    'tip': np.arange(1, 7),
    'total_bill': np.arange(10, 70, 10)
})
df

,time,day,tip,total_bill
0,Lunch,Fri,1,10
1,Lunch,Sat,2,20
2,Lunch,Sun,3,30
3,Dinner,Fri,4,40
4,Dinner,Sat,5,50
5,Dinner,Sun,6,60


`pd.DataFrame.pivot()` 메서드 파라미터:

* `index`: pivoting한 데이터프레임에서, row index로 사용할 컬럼(들)의 이름.
* `columns`: pivoting한 데이터프레임에서, 컬럼 이름으로 사용할 컬럼(들)의 이름.
* `values`: pivoting한 데이터프레임의 각 셀에 채울 값들이 있는 컬럼(들)의 이름.


In [17]:
df.pivot(index='time', columns='day', values='tip')

day,Fri,Sat,Sun
time,,,
Dinner,4,5,6
Lunch,1,2,3


In [18]:
df.pivot(index='day', columns='time', values='tip')

time,Dinner,Lunch
day,,
Fri,4,1
Sat,5,2
Sun,6,3
